In [1]:
from datasets import load_dataset
from sentence_transformers.losses import CosineSimilarityLoss

from setfit import SetFitModel, SetFitTrainer

In [2]:
dataset = load_dataset("nsmc")

Using custom data configuration default
Reusing dataset nsmc (/home/cleaning/.cache/huggingface/datasets/nsmc/default/1.1.0/bfd4729bf1a67114e5267e6916b9e4807010aeb238e4a3c2b95fbfa3a014b5f3)


  0%|          | 0/2 [00:00<?, ?it/s]

In [6]:
num_classes = 2
train_dataset = dataset["train"].shuffle(seed=42).select(range(8 * num_classes))
eval_dataset = dataset["test"]

In [16]:
dataset1 = load_dataset("sst2")

Using custom data configuration default


Generating train split:   0%|          | 0/67349 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/872 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1821 [00:00<?, ? examples/s]

Dataset sst2 downloaded and prepared to /home/cleaning/.cache/huggingface/datasets/sst2/default/2.0.0/9896208a8d85db057ac50c72282bcb8fe755accc671a57dd8059d4e130961ed5. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [15]:
train_dataset['document']

['For Carl.칼 세이건으로 시작해서 칼 세이건으로 끝난다.',
 '모든 면에서 너무 좋고, 특히 동양적이고',
 '달콤한 꿀과 톡쏘는 칠리의 만남',
 '토비 맥과이어가 그립다.',
 '전두엽절제술 ㅠㅠ 넘싫다',
 '재밌게보고있지만 점점식상해지는부분도없지않음',
 '전작의 명성에 똥칠하는 망작',
 '산으로 가는 스토리',
 '이것도영화라고ㅋㅋ',
 '프랑스 교도소 간수 죽여버리고 싶었다',
 '미이케 다케시 영화는 복불복임 ㅋㅋㅋ 개쩌는영화는 진짜 개쩌는데... 이건 영~~',
 '한쪽을 잃은 사람들은 현재와과거에서 상통하는걸 모르는 설탕가게(CJ)영화간부들',
 '유카와 센세~!!최고로 매력적인 캐릭터..후쿠야마 마사하루 말고 그누가 어울릴까..',
 '유사 소재 한국 영화 중 비교적 지루하고 임팩트 없음. 그나마 김윤석의 조연시절 보는 재미와 유지태의 깔끔한 연기를 보는 재미 정도 있음.',
 '진짜 재미있을 것 같은데, 부디 우라나라 애니메이션에 커다란 발전이 있기를.',
 '일본하고 독일을 엮지마라 적어도 일본은 나치스처럼 인종학살은 하지않았다.']

In [18]:
model = SetFitModel.from_pretrained("ddobokki/klue-roberta-base-nli-sts")

model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.


In [19]:
trainer = SetFitTrainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    loss_class=CosineSimilarityLoss,
    metric="accuracy",
    batch_size=16,
    num_iterations=20, # The number of text pairs to generate for contrastive learning
    num_epochs=1, # The number of epochs to use for constrastive learning
    column_mapping={'document': 'text', "label": "label"} # Map dataset columns to text/label expected by trainer
)


In [20]:
trainer.train()
metrics = trainer.evaluate()

Applying column mapping to training dataset
***** Running training *****
  Num examples = 640
  Num epochs = 1
  Total optimization steps = 40
  Total train batch size = 16


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/40 [00:00<?, ?it/s]

Applying column mapping to evaluation dataset


***** Running evaluation *****


In [25]:
import random


In [26]:
eval_dataset

Dataset({
    features: ['id', 'document', 'label'],
    num_rows: 50000
})

In [21]:
idx1 = random.randint(0,len(eval_dataset))
idx2 = random.randint(0,len(eval_dataset))

print(f"text1 :{eval_dataset[idx1]['document']} - label: {eval_dataset[idx1]['label']}")

preds = model(["i loved the spiderman movie!", "pineapple on pizza is the worst 🤮"]) 

{'accuracy': 0.502}